## Data Preparation 1
This code aims to clean the data of bike count with its Monitoring Locations in London

How it works:
1. Check the list of Monitoring Locations
2. Check the sample of bike counts raw data
3. Read all the bike counts raw data, merge the rows into hourly data
4. Join to list of monitoring locations to get the column of borough, funtional, and coordinate
5. Extract List of dates for rainfall data (Data_Preparation_2)

In [1]:
# Import Necessary Package

import pandas as pd
import numpy as np
import os

## List of Monitoring Locations / Points

In [2]:
monitor_loc = pd.read_csv('bike_count/processed/Point_All_cleaned.csv', sep=',')

# Rename 
monitor_loc.rename(columns={'Site ID': 'UnqID'}, inplace=True)
monitor_loc.rename(columns={'Easting (U': 'Easting'}, inplace=True)
monitor_loc.rename(columns={'Northing (': 'Northing'}, inplace=True)

print (monitor_loc.head())

# Note: The Road Type is coming from the raw data: not complete
# Note: Cleaned means raw data (Site ID) already overlayed with the boundary with boundary shapefile

    UnqID                            Location         Borough  Functional  \
0  ML0001  Millbank (south of Thorney Street)     Westminster  01 Central   
1  ML0002                         Bishopsgate  City of London  01 Central   
2  ML0003                    Southwark Bridge       Southwark  01 Central   
3  ML0004               Southwark Bridge Road       Southwark  01 Central   
4  ML0005                       Tooley Street       Southwark  01 Central   

   Road_type Old site I    Easting   Northing   Latitude  Longitude  
0  01 A Road   CENCY001  530251.49  178742.45  51.492628  -0.125204  
1  01 A Road   CENCY002  533362.68  181824.45  51.519599  -0.079254  
2  01 A Road   CENCY003  532334.06  180520.37  51.508123  -0.094551  
3  01 A Road   CENCY004  532052.50  179677.64  51.500613  -0.098927  
4  01 A Road   CENCY005  533031.59  180213.46  51.505200  -0.084629  


## List of Bike Counts Raw Data

Only Check 1 Sample to aggregate/ sum hourly

In [3]:
# check 1 sample of the raw data: year 2015 only central
y2015_central = pd.read_csv('bike_count/raw_data_tfl/2015 Q2 spring (Apr-Jun)-Central.csv', sep=',')
print (y2015_central)

                           Year   UnqID        Date Weather      Time  \
0      2015 Q2 spring (Apr-Jun)  ML0001  04/06/2015     Dry  06:00:00   
1      2015 Q2 spring (Apr-Jun)  ML0001  04/06/2015     Dry  06:15:00   
2      2015 Q2 spring (Apr-Jun)  ML0001  04/06/2015     Dry  06:30:00   
3      2015 Q2 spring (Apr-Jun)  ML0001  04/06/2015     Dry  06:45:00   
4      2015 Q2 spring (Apr-Jun)  ML0001  04/06/2015     Dry  07:00:00   
...                         ...     ...         ...     ...       ...   
51451  2015 Q2 spring (Apr-Jun)  ML0104  15/06/2015     Dry  20:45:00   
51452  2015 Q2 spring (Apr-Jun)  ML0104  15/06/2015     Dry  21:00:00   
51453  2015 Q2 spring (Apr-Jun)  ML0104  15/06/2015     Dry  21:15:00   
51454  2015 Q2 spring (Apr-Jun)  ML0104  15/06/2015     Dry  21:30:00   
51455  2015 Q2 spring (Apr-Jun)  ML0104  15/06/2015     Dry  21:45:00   

           Day Round         Dir Path              Mode  Count  
0      Weekday     A  Northbound  NaN    Private cycles   

In [4]:
# Convert "Date" column to datetime format
y2015_central['Date'] = pd.to_datetime(y2015_central['Date'], format='%d/%m/%Y')

# Convert "Time" column to datetime format
y2015_central['Time'] = pd.to_datetime(y2015_central['Time'], format='%H:%M:%S')

# Extract the hour from the "Time" column
y2015_central['Hour'] = y2015_central['Time'].dt.hour

# Group by "Date," "UnqID," and "Hour" and aggregate the "Count" column
hourly_data_y2015_central = y2015_central.groupby(['UnqID', 'Date', 'Hour'])['Count'].sum().reset_index()

# Display the aggregated hourly data
print(hourly_data_y2015_central)

       UnqID       Date  Hour  Count
0     ML0001 2015-06-04     6     86
1     ML0001 2015-06-04     7    491
2     ML0001 2015-06-04     8    853
3     ML0001 2015-06-04     9    385
4     ML0001 2015-06-04    14     88
...      ...        ...   ...    ...
3374  ML0208 2015-04-30    17     23
3375  ML0208 2015-04-30    18     36
3376  ML0208 2015-04-30    19     18
3377  ML0208 2015-04-30    20     27
3378  ML0208 2015-04-30    21     18

[3379 rows x 4 columns]


For notes: Hour 6 means 06:00 - 06:59

Now, try to iterate to all files.

In [5]:
# List of specific file names to process
file_names = [
    "2015 Q2 spring (Apr-Jun)-Central.csv",
    "2015 Q2 spring (Apr-Jun)-Cycleways.csv",
    "2015 Q2 spring (Apr-Jun)-Inner.csv",
    "2015 Q2 spring (Apr-Jun)-Outer.csv",
    "2016 Q2 spring (Apr-Jun)-Central.csv",
    "2016 Q2 spring (Apr-Jun)-Cycleways.csv",
    "2016 Q2 spring (Apr-Jun)-Inner.csv",
    "2016 Q2 spring (Apr-Jun)-Outer.csv",
    "2017 Q2 spring (Apr-Jun)-Central.csv",
    "2017 Q2 spring (Apr-Jun)-Cycleways.csv",
    "2017 Q2 spring (Apr-Jun)-Inner.csv",
    "2017 Q2 spring (Apr-Jun)-Outer.csv",
    "2018 Q2 spring (Apr-Jun)-Central.csv",
    "2018 Q2 spring (Apr-Jun)-Cycleways.csv",
    "2018 Q2 spring (Apr-Jun)-Inner.csv",
    "2018 Q2 spring (Apr-Jun)-Outer.csv",
    "2019 Q2 spring (Apr-Jun)-Central.csv",
    "2019 Q2 spring (Apr-Jun)-Cycleways.csv",
    "2019 Q2 spring (Apr-Jun)-Inner.csv",
    "2019 Q2 spring (Apr-Jun)-Outer.csv",
    "2020 Q2 spring (synthetic)-Central.csv",
    "2020 Q2 spring (synthetic)-Inner.csv",
    "2020 Q2 spring (synthetic)-Outer.csv",
    "2021 Q2 spring (Apr-Jun)-Central.csv",
    "2021 Q2 spring (Apr-Jun)-Cycleways.csv",
    "2021 Q2 spring (Apr-Jun)-Inner.csv",
    "2021 Q2 spring (Apr-Jun)-Outer.csv",
    "2022 W1 spring-Central.csv",
    "2022 W1 spring-Cycleways.csv",
    "2022 W1 spring-Inner-Part1.csv",
    "2022 W1 spring-Inner-Part2.csv",
    "2022 W1 spring-Outer.csv",
    "2023 W1 spring-Central.csv",
    "2023 W1 spring-Cycleways.csv",
    "2023 W1 spring-Inner-Part1.csv",
    "2023 W1 spring-Inner-Part2.csv",
    "2023 W1 spring-Outer.csv",
    "2024 W1 spring-Central.csv",
    "2024 W1 spring-Cycleways.csv",
    "2024 W1 spring-Inner-Part1.csv",
    "2024 W1 spring-Inner-Part2.csv",
    "2024 W1 spring-Outer.csv"
]

# Year 2020 does not have Cycleways data (incomplete), and the data is synthetic, should I still used the data?

In [6]:
# Base path of the files
base_path = 'bike_count/raw_data_tfl'

# List to store processed data
all_hourly_data = []

# Loop through the specified files
for file_name in file_names:
    # Construct the full file path
    file_path = f"{base_path}\\{file_name}"
    
    # Read the CSV file
    data = pd.read_csv(file_path, sep=',')

    # Extract the year part from the "year" column and store it in a new column "wave"
    data['wave'] = data['Year'].str.extract(r'(\d{4})')

    # Exclude rows where "Mode" is "E-scooters" or "Pedestrian"
    data = data[~data['Mode'].isin(['E-scooters', 'Pedestrians'])]
    
    # Convert "Date" column to datetime format
    data['Date'] = pd.to_datetime(data['Date'], format='%d/%m/%Y')

    # Extract the year from "Date" column
    data['Year'] = data['Date'].dt.year
    
    # Convert "Time" column to datetime format
    data['Time'] = pd.to_datetime(data['Time'], format='%H:%M:%S')
    
    # Extract the hour from the "Time" column
    data['Hour'] = data['Time'].dt.hour
    
    # Group by "Date", "UnqID", and "Hour" and aggregate the "Count" column
    # Weather is not added, because the same hour in one day could have 2 types of weather description. It will be replaced by continues data later on.  
    hourly_data = data.groupby(['UnqID', 'wave', 'Year', 'Date', 'Hour'])['Count'].sum().reset_index()
    
    # Add a column to indicate the source file
    hourly_data['SourceFile'] = file_name
    
    # Append the processed data to the list
    all_hourly_data.append(hourly_data)

# Combine all the processed data into a single DataFrame
final_hourly_data = pd.concat(all_hourly_data, ignore_index=True)

# Display the aggregated hourly data
print(final_hourly_data)

C:\Users\arthu\AppData\Local\Temp\ipykernel_25484\2768977537.py:13: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, sep=',')
C:\Users\arthu\AppData\Local\Temp\ipykernel_25484\2768977537.py:13: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, sep=',')


         UnqID  wave  Year       Date  Hour  Count  \
0       ML0001  2015  2015 2015-06-04     6     86   
1       ML0001  2015  2015 2015-06-04     7    491   
2       ML0001  2015  2015 2015-06-04     8    853   
3       ML0001  2015  2015 2015-06-04     9    385   
4       ML0001  2015  2015 2015-06-04    14     88   
...        ...   ...   ...        ...   ...    ...   
249492  ML1451  2024  2024 2024-04-18    17     29   
249493  ML1451  2024  2024 2024-04-18    18     47   
249494  ML1451  2024  2024 2024-04-18    19     35   
249495  ML1451  2024  2024 2024-04-18    20     11   
249496  ML1451  2024  2024 2024-04-18    21      6   

                                  SourceFile  
0       2015 Q2 spring (Apr-Jun)-Central.csv  
1       2015 Q2 spring (Apr-Jun)-Central.csv  
2       2015 Q2 spring (Apr-Jun)-Central.csv  
3       2015 Q2 spring (Apr-Jun)-Central.csv  
4       2015 Q2 spring (Apr-Jun)-Central.csv  
...                                      ...  
249492              20

In [7]:
# Filter rows where 'wave' and 'Year' are different
mismatched_rows = final_hourly_data[final_hourly_data['wave'] != final_hourly_data['Year']]

# Display the mismatched rows
print(mismatched_rows)

         UnqID  wave  Year       Date  Hour  Count  \
0       ML0001  2015  2015 2015-06-04     6     86   
1       ML0001  2015  2015 2015-06-04     7    491   
2       ML0001  2015  2015 2015-06-04     8    853   
3       ML0001  2015  2015 2015-06-04     9    385   
4       ML0001  2015  2015 2015-06-04    14     88   
...        ...   ...   ...        ...   ...    ...   
249492  ML1451  2024  2024 2024-04-18    17     29   
249493  ML1451  2024  2024 2024-04-18    18     47   
249494  ML1451  2024  2024 2024-04-18    19     35   
249495  ML1451  2024  2024 2024-04-18    20     11   
249496  ML1451  2024  2024 2024-04-18    21      6   

                                  SourceFile  
0       2015 Q2 spring (Apr-Jun)-Central.csv  
1       2015 Q2 spring (Apr-Jun)-Central.csv  
2       2015 Q2 spring (Apr-Jun)-Central.csv  
3       2015 Q2 spring (Apr-Jun)-Central.csv  
4       2015 Q2 spring (Apr-Jun)-Central.csv  
...                                      ...  
249492              20

In [8]:
# Save the aggregated data to a new CSV file
final_hourly_data.to_csv('bike_count/processed/aggregated_hourly_data.csv', index=False)

In [9]:
# Check for duplicate rows
duplicate_rows = final_hourly_data[final_hourly_data.duplicated()]

# Display the duplicate rows
print("Duplicate rows:")
print(duplicate_rows)

# Count the number of duplicate rows
num_duplicates = duplicate_rows.shape[0]
print(f"Number of duplicate rows: {num_duplicates}")

# If needed, save duplicate rows to a CSV file for review
#if num_duplicates > 0:
#    duplicate_rows.to_csv('duplicate_rows.csv', index=False)

Duplicate rows:
Empty DataFrame
Columns: [UnqID, wave, Year, Date, Hour, Count, SourceFile]
Index: []
Number of duplicate rows: 0


## Join to Monitoring Locations/ Points

In [10]:
# Perform a left join between hourly_data and monitor_loc on the "UnqID" column
merged_hourly_data = pd.merge(final_hourly_data, monitor_loc, on='UnqID', how='left')

# Reorder the columns to make 'UnqID' the first column
merged_hourly_data = merged_hourly_data[['UnqID', 'wave', 'Year', 'Date', 'Hour', 'Count', 'Borough', 'Functional', 'Latitude', 'Longitude', 'Easting', 'Northing']]

# Display the merged table
print(merged_hourly_data)

         UnqID  wave  Year       Date  Hour  Count      Borough  Functional  \
0       ML0001  2015  2015 2015-06-04     6     86  Westminster  01 Central   
1       ML0001  2015  2015 2015-06-04     7    491  Westminster  01 Central   
2       ML0001  2015  2015 2015-06-04     8    853  Westminster  01 Central   
3       ML0001  2015  2015 2015-06-04     9    385  Westminster  01 Central   
4       ML0001  2015  2015 2015-06-04    14     88  Westminster  01 Central   
...        ...   ...   ...        ...   ...    ...          ...         ...   
249492  ML1451  2024  2024 2024-04-18    17     29        Brent    03 Outer   
249493  ML1451  2024  2024 2024-04-18    18     47        Brent    03 Outer   
249494  ML1451  2024  2024 2024-04-18    19     35        Brent    03 Outer   
249495  ML1451  2024  2024 2024-04-18    20     11        Brent    03 Outer   
249496  ML1451  2024  2024 2024-04-18    21      6        Brent    03 Outer   

         Latitude  Longitude    Easting   Northing 

In [11]:
# Save the merged DataFrame to a CSV file in your project directory
merged_hourly_data.to_csv('bike_count/processed/merged_hourly_data.csv', index=False)

In [14]:
output_dir = 'bike_count/processed/yearly_data'
os.makedirs(output_dir, exist_ok=True)

# Iterate over each unique year in the 'Year' column
unique_years = merged_hourly_data['Year'].unique()

for year in unique_years:
    # Filter the DataFrame for the current year
    yearly_data = merged_hourly_data[merged_hourly_data['Year'] == year]
    
    # Create a file name for the current year
    file_name = f'{output_dir}/bike_count_{year}.csv'
    
    # Save the filtered data to a CSV file
    yearly_data.to_csv(file_name, index=False)

print(f"Data successfully split into {len(unique_years)} files and saved to {output_dir}.")

Data successfully split into 11 files and saved to bike_count/processed/yearly_data.


## Extract List of Dates for Rainfall Data (Data_Preparation_2)

In [12]:
# Extract unique dates
unique_dates_per_year = merged_hourly_data[['wave', 'Year', 'Date']].drop_duplicates().sort_values(by=['wave', 'Year', 'Date'])

# Reset the index for a clean output
unique_dates_per_year = unique_dates_per_year.reset_index(drop=True)

# Display the vertical list
print(unique_dates_per_year)

# Optionally save to a CSV file
unique_dates_per_year.to_csv('bike_count/processed/unique_dates.csv', index=False)

      wave  Year       Date
0     2015  2014 2014-09-01
1     2015  2014 2014-09-02
2     2015  2014 2014-09-08
3     2015  2014 2014-09-09
4     2015  2014 2014-09-10
...    ...   ...        ...
1039  2024  2024 2024-07-11
1040  2024  2024 2024-07-16
1041  2024  2024 2024-07-17
1042  2024  2024 2024-07-18
1043  2024  2024 2024-07-23

[1044 rows x 3 columns]
